In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
topo_data = xr.open_dataset(r'./dem_nc.nc')
topo_data

<xarray.Dataset>
Dimensions:  (lat: 1920, lon: 5064)
Coordinates:
  * lat      (lat) float64 42.9 42.9 42.9 42.9 42.9 ... 44.5 44.5 44.5 44.5 44.5
  * lon      (lon) float64 -80.06 -80.06 -80.06 -80.06 ... -75.84 -75.84 -75.84
Data variables:
    crs      |S1 ...
    Band1    (lat, lon) float32 ...
Attributes:
    GDAL_AREA_OR_POINT:  Area
    GDAL_DataType:       Generic
    Conventions:         CF-1.5
    GDAL:                GDAL 3.4.1, released 2021/12/27
    history:             Sat Dec 21 21:50:22 2024: GDAL CreateCopy( dem_nc.nc...

In [3]:
def temp_interp(h):
    if h>100.0:
        temp = 5.5
    else:
        temp = 5.5+(100.0-h)/100.0*(10.8-5.5)
    return (temp)

In [ ]:
layer_num = 16
lon_out = np.arange(-80.5, -75.5+0.001, 0.01)
lat_out = np.arange(45.0, 42.499, -0.01)
lat_count = len(lat_out)
lon_count = len(lon_out)

temp_top = 10.80

import os
os.system(r"rm -rf ./temp_inter")
save_file = r'./temp_inter'

with open(save_file, "a") as f:
    # no "preamber" for now
    # loop over the layers
    for layer in range(layer_num):
        output_tot = np.zeros((1, lon_count))
        # start with the bottom layer
        f.write('"tstep" 0 "item" 1 "layer" %s \n' % str(layer))
        for j in range(lat_count):
            for k in range(lon_count):
                depth = -1.0*topo_data.Band1.sel( lon = lon_out[k], lat = lat_out[j],method='nearest')
                if depth<0:
                    # land
                    output_tot[:, k] = temp_top
                else:
                    # water
                    # make sure the top layer is temp_top 
                    output_tot[:, k] = temp_interp(depth/layer_num*(layer_num-layer-1))
                    if (layer==(layer_num-1)):
                        output_tot[:, k] = temp_top
            # output_tot = np.transpose(output_tot)
            np.savetxt(f, output_tot, fmt='%1.4f')
            # f.write('\n')
        f.write('\n')
        print('Layer'+': '+str(layer))

f.close()

Layer: 0
Layer: 1
